In [1]:
import geopandas as gpd
import stormcatchments as sc
import fiona
import matplotlib.pyplot as plt
import contextily as cx
import numpy as np
import pandas as pd
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from shapely.geometry import box, LineString, Point,MultiPoint
import os
import sys
import fiona
from scipy.spatial import cKDTree
from shapely import wkt
import time
from rtree import index
sys.path.append(os.path.realpath('..'))


#see all columns in tables
pd.set_option('display.max_columns', None)

In [2]:
muni = 'Natick'


#read in municipalities, select the one of interest
#municipalities
munis_fp = "K:\\\DataServices\\Datasets\\Boundaries\\Spatial\\mapc_towns_poly.shp"
munis = gpd.read_file(munis_fp)


#select just the muni 
muni_shp = munis.loc[munis['municipal'] == muni]

In [3]:
drainage_network_path  = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Data\\Spatial\\Drainage_Network"


In [118]:
import rasterio 
from rasterio.mask import mask

inRas = 'I:\\Elevation\\Lidar\\2023_MassGIS\\LiDAR_DEM_INT_16bit.tiff'
outRas = 'I:\\Elevation\\Lidar\\2023_MassGIS\\' + muni + '_LiDAR_DEM_INT_16bit.tiff'
muni_shp_buffer = muni_shp.buffer(1000)

#clip the dem to the muni, save to I drive folder
with rasterio.open(inRas) as src:
    #update crs
    src.meta.update({
        'crs': muni_shp.crs
        })
    out_image, out_transform= mask(src,muni_shp_buffer.geometry,crop=True)
    out_meta=src.meta.copy() # copy the metadata of the source DEM

    
out_meta.update({
    "driver":"Gtiff",
    "height":out_image.shape[1], # height starts with shape[1]
    "width":out_image.shape[2], # width starts with shape[2]
    "transform":out_transform
})
              
with rasterio.open(outRas,'w',**out_meta) as dst:
    dst.write(out_image)

In [4]:
#natick
natick_path = drainage_network_path + '\\Natick Stormwater\\mrged.gdb'
natick_pts = gpd.read_file (natick_path, layer = 'd_nodes')
natick_lines = gpd.read_file (natick_path, layer = 'd_pipes')

#reproject all to mass mainland
mass_mainland_crs = "EPSG:26986"
natick_pts = natick_pts.to_crs(mass_mainland_crs).reset_index()
natick_lines = natick_lines.to_crs(mass_mainland_crs).reset_index()

#explode to get rid of multigeometry
natick_lines = natick_lines.explode()


C:\Users\rbowers\AppData\Local\Temp\ipykernel_12112\886053387.py:12: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  natick_lines = natick_lines.explode()


In [5]:
#define 'sources' and 'sinks' 
#sinks corresponds to catch basins and culvert inlets
#sources corresponds to outfalls and culvert outlets
source_types = ['CB', 'CI']
sink_types = ['OF', 'CO']

#add boolean 'IS_SOURCE' and 'IS_SINK' field (for storm catchments package)
natick_pts['IS_SOURCE'] = np.where(natick_pts['TYPE'].isin(source_types), True, False)
natick_pts['IS_SINK'] = np.where(natick_pts['TYPE'].isin(sink_types), True, False)

#these had some funny indeces issues that we're fixing
natick_pts = natick_pts.reset_index(drop=True)
natick_lines = natick_lines.reset_index(drop=True)

#run storm catchments network 
net = sc.Network(natick_lines, 
                 natick_pts, 
                 coord_decimals=2)


In [18]:
from src.features.storm_catch import *

net = snap_points(net, 30)
net.resolve_directions(method='from_sources', verbose=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
#preprocess the dem and initialize the "Delineate" object 
grid, fdir, acc = sc.terrain.preprocess_dem(outRas)
grid_epsg = 26986
delin = sc.Delineate(net, grid, fdir, acc, grid_epsg)

c:\Users\rbowers\Anaconda3\envs\ms4_env\lib\site-packages\pysheds\io.py:142: UserWarning: No `nodata` value detected. Defaulting to 0.
  warnings.warn('No `nodata` value detected. Defaulting to 0.')
c:\Users\rbowers\Anaconda3\envs\ms4_env\lib\site-packages\pysheds\io.py:142: UserWarning: No `nodata` value detected. Defaulting to 0.
  warnings.warn('No `nodata` value detected. Defaulting to 0.')


In [122]:
natick_source_points = natick_pts.loc[natick_pts['IS_SOURCE'] == True]
natick_sink_points = natick_pts.loc[natick_pts['IS_SINK'] == True]


dest = gpd.GeoDataFrame(columns=['index', 'TYPE', 'geometry'], 
                        geometry='geometry', 
                        crs='EPSG:26986')


for row in natick_sink_points.itertuples(): #looping over pts
    coords = (row.geometry.x, row.geometry.y)
    print(row.index)
    catchment = sc.delineate.get_catchment(coords, 
                                           grid, 
                                           fdir, 
                                           acc, 
                                           grid_epsg, 
                                           acc_thresh=1000)
    catchment['index'] = row.index
    catchment['TYPE'] = row.TYPE
    dest = pd.concat([dest, catchment])

dest.head()

4
5
11
5036
5037
5038
5039
5040
5041
5042
5043
5044
5045
5046
5049
5052
5053
5054
5058
5059
5061
5065
5066
5067
5068
5069
5070
5071
5072
5073
5074
5075
5076
5077
5078
5080
5083
5084
5085
5086
5087
5088
5090
5091
5092
5093
5096
5098
5099
5100
5101
5103
5104
5105
5107
5108
5109
5110
5111
5112
5113
5114
5115
5116
5117
5118
5119
5120
5121
5122
5123
5124
5125
5126
5127
5129
5130
5131
5132
5138


MemoryError: Allocation failed (probably too large).

In [121]:
#export to a folder with the name of the town
#or do we first calculate the nutrient load of each catchment area? 
dest.head()

,index,TYPE,geometry
